In [1]:
import h5py
import os

from keras.applications import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

Using Theano backend.


In [11]:
#path to training data
DATA_PATH = 'C:\\Users\\chapanda\\Documents\\cp\\Misc\\Scripts\\practise\\dogcats\\courses-master\\deeplearning1\\nbs\\data\\fish\\train'

#Number of clusters for K-Means
N_CLUSTS = 250

#Number of clusters used for validation
N_VAL_CLUSTS = 50

SEED = 42
np.random.seed(SEED)

IMG_WIDTH = 640
IMG_HEIGHT = 360

In [4]:
base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (3 , IMG_HEIGHT, IMG_WIDTH))
model = Model(input = base_model.input, output = base_model.get_layer('block4_pool').output)

58826752/58889096 [============================>.] - ETA: 0s

In [5]:
def preprocess_image(path):
    img = image.load_img(path, target_size = (IMG_HEIGHT, IMG_WIDTH))
    arr = image.img_to_array(img)
    arr = np.expand_dims(arr, axis = 0)
    return preprocess_input(arr)

In [12]:
subsample = []
for fish in os.listdir(DATA_PATH):
    if(os.path.isfile(os.path.join(DATA_PATH, fish))): 
        continue
    subsample_class = [os.path.join(DATA_PATH, fish, fn) for 
                       fn in os.listdir(os.path.join(DATA_PATH, fish))]
    subsample += subsample_class
len(subsample)

In [14]:
%%time
preprocessed_images = np.vstack([preprocess_image(fn) for fn in subsample])
vgg_features = model.predict(preprocessed_images)
vgg_features = vgg_features.reshape(len(subsample), -1)

ERROR (theano.gof.opt): Optimization failure due to: local_abstractconv_check
ERROR (theano.gof.opt): node: AbstractConv2d{border_mode='half', subsample=(1, 1), filter_flip=True, imshp=(None, None, None, None), kshp=(64, 3, 3, 3)}(input_1, block1_conv1_W)
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "C:\Users\chapanda\AppData\Local\Continuum\Anaconda2n\lib\site-packages\theano\gof\opt.py", line 1772, in process_node
    replacements = lopt.transform(node)
  File "C:\Users\chapanda\AppData\Local\Continuum\Anaconda2n\lib\site-packages\theano\tensor\nnet\opt.py", line 402, in local_abstractconv_check
    node.op.__class__.__name__)
AssertionError: AbstractConv2d Theano optimization failed: there is no implementation available supporting the requested options. Did you exclude both "conv_dnn" and "conv_gemm" from the optimizer? If on GPU, is cuDNN available and does the GPU support it? If on CPU, do you have a BLAS library installed Th

AssertionError: AbstractConv2d Theano optimization failed: there is no implementation available supporting the requested options. Did you exclude both "conv_dnn" and "conv_gemm" from the optimizer? If on GPU, is cuDNN available and does the GPU support it? If on CPU, do you have a BLAS library installed Theano can link against?

In [ ]:
%%time
km = KMeans(n_clusters = N_CLUSTS, n_jobs = -1)
clust_preds = km.fit_predict(StandardScaler().fit_transform(vgg_features))

In [ ]:
val_clusters = np.random.choice(range(N_CLUSTS), N_VAL_CLUSTS, replace = False)
val_sample = np.array(subsample)[np.in1d(clust_preds, val_clusters)]